<a href="https://colab.research.google.com/github/joelwwiggins/car_ml/blob/main/Copy_of_notebook_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
#import csv from path
#drop first row and column
df1 = pd.read_csv('CSVLog_20221130_091051.csv', skiprows=1)


In [4]:
#drop first column
df1_col=df1.iloc[:,1:]
df=df1_col

In [5]:
df.head()

,Latitude (deg),Longitude (deg),Vehicle speed (MPH),Instant fuel economy (MPG),Total fuel economy (MPG),Fuel rate (gal/hr),Instant CO2 rate (lb/mile),Total CO2 (lbs),CO2 flow (lb/min),Accel X (ft/s²),...,Average Trip CO2 Rate (lb/mile),Max Speed (MPH),Seconds Idling (sec),Idling Count,Hard Accel Count,Hard Brake Count,Trip Duration (min),Trip Fuel Economy (MPG),Trip Fuel (gal),Trip Distance (miles)
0,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
1,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.294891,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
2,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.317436,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
3,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.317436,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
4,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.346312,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0


In [6]:
#convert to numpy array
df_np=df.to_numpy()

#train test split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_np, test_size=0.2, random_state=42)

#normalize the x_train
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#reshape the data to 3d
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1],1))

#use a 1dconv model to predict the next value
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D 
from keras.layers import Dropout
from keras.layers import LSTM
#import resnet 50 model

from keras.preprocessing import image
from keras.models import Model


model_shape = X_train_scaled.shape



nn_model = tf.keras.models.Sequential()
nn_model.build(input_shape=model_shape)

nn_model.add(tf.keras.layers.Conv1D(512,3,activation="relu",padding='same',input_shape=model_shape[1:]))

nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))


nn_model.add(tf.keras.layers.Flatten())
nn_model.add(tf.keras.layers.Dense(units=model_shape[1], activation='linear'))




nn_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_mae',patience=10)
# checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath ="/content/sample_data")
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)
cb_list = [es, tbCallBack]
nn_model.fit(X_train, X_train, epochs=30,validation_split=.1,shuffle=True,callbacks=cb_list)
#normalize the x_test
X_test_scaled = scaler.fit_transform(X_test)
nn_model.evaluate(X_test_scaled, X_test_scaled)



nn_model.summary()

Epoch 1/30
845/845 [==============================] - 26s 31ms/step - loss: 205557.2969 - mae: 33.1926 - mse: 205557.2969 - val_loss: 1883.9034 - val_mae: 28.6745 - val_mse: 1883.9034
Epoch 2/30
845/845 [==============================] - 27s 32ms/step - loss: 1282.8239 - mae: 25.2881 - mse: 1282.8239 - val_loss: 878.0887 - val_mae: 21.5272 - val_mse: 878.0887
Epoch 3/30
845/845 [==============================] - 26s 31ms/step - loss: 1122.2577 - mae: 24.2832 - mse: 1122.2577 - val_loss: 996.1411 - val_mae: 25.2567 - val_mse: 996.1411
Epoch 4/30
845/845 [==============================] - 27s 32ms/step - loss: 872.3154 - mae: 20.8117 - mse: 872.3154 - val_loss: 474.7344 - val_mae: 17.0838 - val_mse: 474.7344
Epoch 5/30
845/845 [==============================] - 27s 32ms/step - loss: 711.1953 - mae: 19.0832 - mse: 711.1953 - val_loss: 1501.4706 - val_mae: 29.7122 - val_mse: 1501.4706
Epoch 6/30
845/845 [==============================] - 28s 33ms/step - loss: 540.3541 - mae: 16.9282 - mse:

In [7]:
#save the model
nn_model.save('model.h5')

In [8]:
import plotly.express as px

reconstructions_test = nn_model.predict(X_test_scaled)
test_loss = tf.keras.losses.mae(reconstructions_test, X_test_scaled)


df_loss_test = pd.DataFrame(test_loss)
fig = px.histogram(df_loss_test, x=df_loss_test.iloc[:,0], nbins=50)
fig.update_layout(yaxis_type="log")
fig.show()